In [1]:
import pandas as pd

In [2]:
df = pd.read_csv("../processed-datasets/external-feature-dataset.csv")

In [3]:
df = df.drop(["Unnamed: 0", "domain"], axis=1)
df.head()

,class,url,status_code,html
0,0,https://crowdyfan.com/,200,b'eNrtfet220bS4G/rHL9Dh5kxpS8CCYB3ymI+WZJtzcS2...
1,0,https://dreamaways.com/,403,b'eNqzySjJzbHj5bLJSE1MsbMpySzJSbUzMTBWcMsvSspM...
2,0,https://www.worldtravelserver.com/,200,b'eNrsvXuX4zaSL/h396fgpI+PK68lFfWWUq7ccdvt7t6e...
3,0,https://www.baarty.com/,200,b'eNrtfdtyG0ey4LMVoX8oY+KMpDho3HiXSXgoirY5K1k8...
4,0,https://ladamotors63.ru/,200,b'eNrtPf1zG0WWP1t/RWfudmMdHo0+bMtyLLMkhKvUZoFa...


In [4]:
df.groupby('status_code').count()

,class,url,html
status_code,,,
200,52531,52531,52531
201,4,4,4
202,2,2,2
203,3,3,3
204,363,363,363
301,1,1,1
302,2,2,2
400,1029,1029,1029
401,112,112,112


In [5]:
df.groupby('class').count()

,url,status_code,html
class,,,
0,43609,43609,43609
1,30589,30589,30589


### Extract metadata and page title from page information

In [28]:
from multiprocessing import freeze_support, Pool, Value
import zlib
import base64
import sys
from bs4 import BeautifulSoup

count = 0

def extractor(row):
    str_compressed = row["html"]
    global count
    
    metadata = {
        "description": "",
        "og_description": "",
        "og_title": ""
    }

    
    soup = BeautifulSoup(zlib.decompress(base64.b64decode(str_compressed[2:-1])), "html.parser")
    
    
    title = soup.head.title.text if soup.head and soup.head.title else "None"
    
    metadata["description"] = soup.find("meta", property="description")
    metadata["og_description"] = soup.find("meta", property="og:description")
    metadata["og_title"] = soup.find("meta", property="og:title")
    
    for item in metadata:
        if metadata[item] != None:
            if "content" in metadata[item]:
                metadata[item] = metadata[item]["content"] 
            else:
                metadata[item] =""
        else:
            metadata[item] =""
    
    metadata[title] = title
    
    count += 1
    print("Status:", count , end="\r", flush=True)
    return title, metadata["description"], metadata["og_title"], metadata["og_description"]


In [29]:
tdf = df.head(500)

In [30]:
import swifter

In [31]:
tdf[["title", "meta_desc", "og_title", "og_desc"]] = tdf.apply(extractor, axis = 1, result_type ='expand')

/home/elliot/.local/lib/python3.10/site-packages/pandas/core/frame.py:3641: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[k1] = value[k2]


In [32]:
tdf.head()

,class,url,status_code,html,title,meta_desc,og_title,og_desc
0,0,https://crowdyfan.com/,200,b'eNrtfet220bS4G/rHL9Dh5kxpS8CCYB3ymI+WZJtzcS2...,"Crowdyfan | Sports News, Nutrition & More",,,
1,0,https://dreamaways.com/,403,b'eNqzySjJzbHj5bLJSE1MsbMpySzJSbUzMTBWcMsvSspM...,403 Forbidden,,,
2,0,https://www.worldtravelserver.com/,200,b'eNrsvXuX4zaSL/h396fgpI+PK68lFfWWUq7ccdvt7t6e...,"London, travel agencies, United Kingdom, weath...",,,
3,0,https://www.baarty.com/,200,b'eNrtfdtyG0ey4LMVoX8oY+KMpDho3HiXSXgoirY5K1k8...,Soluciones Para Bares - Baarty.com,,,
4,0,https://ladamotors63.ru/,200,b'eNrtPf1zG0WWP1t/RWfudmMdHo0+bMtyLLMkhKvUZoFa...,Lada Motors,,,


In [34]:
tdf[""].isna()

,class,url,status_code,html,title,meta_desc,og_title,og_desc
0,False,False,False,False,False,False,False,False
1,False,False,False,False,False,False,False,False
2,False,False,False,False,False,False,False,False
3,False,False,False,False,False,False,False,False
4,False,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...
495,False,False,False,False,False,False,False,False
496,False,False,False,False,False,False,False,False
497,False,False,False,False,False,False,False,False
498,False,False,False,False,False,False,False,False
